In [1]:
import cv2
import numpy as np

src = "template/pattern_cropped.png"
out = "template/pattern_cropped_swapped.png"

img = cv2.imread(src, cv2.IMREAD_UNCHANGED)

# Split alpha
bgr = img[:, :, :3]
alpha = img[:, :, 3]

hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)

# Binary masks for red and white areas
lower_red1, upper_red1 = np.array([0,   80, 50], np.uint8), np.array([10, 255,255], np.uint8)
lower_red2, upper_red2 = np.array([170, 80, 50], np.uint8), np.array([180,255,255], np.uint8)
mask_red   = cv2.inRange(hsv, lower_red1, upper_red1) | cv2.inRange(hsv, lower_red2, upper_red2)

lower_white, upper_white = np.array([0,0,200], np.uint8), np.array([180,40,255], np.uint8)
mask_white = cv2.inRange(hsv, lower_white, upper_white)

# ---- Feather edges ----
def soft(mask, sigma=1.2, gain=1.2):
    m = mask.astype(np.float32) / 255.0
    m = cv2.GaussianBlur(m, (0,0), sigmaX=sigma, sigmaY=sigma)   # feather
    m *= gain                                                     # slightly expand coverage
    return np.clip(m, 0, 1)

soft_red   = soft(mask_red,   sigma=1.0, gain=1.2)
soft_white = soft(mask_white, sigma=1.0, gain=1.2)
soft_white[mask_red > 0] = 0
soft_red[mask_white > 0] = 0

# Red tone from the image
red_pixels = bgr[mask_red > 0]
red_color = (np.median(red_pixels, axis=0).astype(np.uint8) if red_pixels.size
             else np.array([0,0,255], np.uint8))  # BGR fallback

# Targets
WHITE = np.array([255,255,255], np.uint8)
RED   = red_color

# ---- Blend ----
out_bgr = bgr.copy()
# red → white
out_bgr = (out_bgr*(1 - soft_red[...,None]) + WHITE[None,None,:]*soft_red[...,None]).astype(np.uint8)
# white → red
out_bgr = (out_bgr*(1 - soft_white[...,None]) + RED[None,None,:]*soft_white[...,None]).astype(np.uint8)

# Reattach alpha and save
out_img = cv2.merge([out_bgr[:,:,0], out_bgr[:,:,1], out_bgr[:,:,2], alpha]) 
cv2.imwrite(out, out_img)
print("Saved:", out)



Saved: template/pattern_cropped_swapped.png
